# Tutorial 2 (Enhanced): Tensor Basics — Shapes, Types, Devices, Manipulations

In this enhanced tutorial you will learn:
- Creating 0D–4D tensors
- Inspecting shape, dtype, and device
- Reshape, permute, squeeze/unsqueeze, broadcasting
- Basic math and autograd
- Save/Load locally and via Tensorus API (demo fallback)

In [ ]:
# Lightweight install cell
import sys, subprocess, pkgutil
for p in ['numpy','torch','matplotlib','seaborn','requests']:
    if pkgutil.find_loader(p) is None:
        subprocess.check_call([sys.executable,'-m','pip','install',p])
print('✅ Dependencies ready')

In [ ]:
# Setup
import torch, numpy as np, requests
import matplotlib.pyplot as plt, seaborn as sns
sns.set_theme(style='whitegrid')
API='http://127.0.0.1:7860'
def server_ok():
    try: return requests.get(f'{API}/health', timeout=2).status_code==200
    except: return False
SERVER=server_ok(); print('📡 Tensorus:', '✅ Connected' if SERVER else '⚠️ Demo Mode')
device='cuda' if torch.cuda.is_available() else 'cpu'; device

## 0D–4D Overview

In [ ]:
scalar=torch.tensor(3.14)
vec=torch.arange(5)
mat=torch.randn(3,4)
img=torch.rand(32,32,3)
batch=torch.rand(8,32,32,3)
print('scalar', tuple(scalar.shape), scalar.dtype)
print('vec', tuple(vec.shape))
print('mat', tuple(mat.shape))
print('img', tuple(img.shape))
print('batch', tuple(batch.shape))

## Devices and dtypes

In [ ]:
x=torch.randn(2,3, device=device, dtype=torch.float32)
y=x.to(torch.float16)
print('x', x.device, x.dtype)
print('y', y.device, y.dtype)
if device=='cuda':
    z=x.to('cpu')
    print('moved back to CPU:', z.device)

## Common Manipulations: view/reshape, permute, squeeze/unsqueeze, broadcasting

In [ ]:
a=torch.arange(24).reshape(2,3,4)
print('a', a.shape)
b=a.view(6,4)  # same data, different view
c=a.reshape(4,6) # may allocate
p=a.permute(2,0,1) # reorder dims
s=torch.randn(1,3,1)
squeezed=s.squeeze()
unsq=squeezed.unsqueeze(0).unsqueeze(-1)
print('b', b.shape, 'c', c.shape, 'p', p.shape)
print('s', s.shape, 'squeezed', squeezed.shape, 'unsq', unsq.shape)
# Broadcasting demo
x=torch.randn(10,1); y=torch.randn(1,5); xy=x+y  # (10,5) via broadcasting
xy.shape

## Basic Math and Autograd

In [ ]:
w=torch.randn(3, requires_grad=True)
b=torch.randn(1, requires_grad=True)
inp=torch.randn(5,3)
target=torch.randn(5,1)
pred=inp@w.unsqueeze(1)+b  # linear
loss=((pred-target)**2).mean()
print('loss:', loss.item())
loss.backward()
print('w.grad', w.grad)
print('b.grad', b.grad)

## Save/Load Locally (PyTorch)

In [ ]:
import tempfile, os
tmp=tempfile.NamedTemporaryFile(delete=False, suffix='.pt').name
torch.save({'w':w,'b':b}, tmp)
loaded=torch.load(tmp)
print('loaded keys:', list(loaded.keys()))
os.unlink(tmp)

## Store & Retrieve with Tensorus (demo fallback)

In [ ]:
payload={'tensor_data': img.tolist(), 'metadata': {'name':'sample_image','shape':list(img.shape)}}
if SERVER:
    try:
        r=requests.post(f'{API}/api/v1/tensors', json=payload, timeout=5)
        tid=r.json().get('tensor_id','demo_tensor')
        rr=requests.get(f'{API}/api/v1/tensors/{tid}', timeout=5)
        print('Stored and retrieved:', tid, '| status:', rr.status_code)
    except Exception as e:
        print('API error, demo mode:', e)
        print('Demo retrieved shape:', tuple(img.shape))
else:
    print('Demo stored tensor_id: demo_tensor | Demo retrieved shape:', tuple(img.shape))

## Next Steps
- Try different dtypes (e.g., float16) and devices.
- Explore `einsum` for expressive math.
- Proceed to Tutorial 3 for hybrid search across text and tensor math.